In [1]:
from datasets import load_dataset
import torch
from transformers import AutoTokenizer, AutoModelForSequenceClassification, TrainingArguments, Trainer

import evaluate
import numpy as np
from transformers import DataCollatorWithPadding
from transformers import DefaultDataCollator

C:\Users\jai54\anaconda3\envs\cuda_env\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
dataset = load_dataset("csv", data_files="AI_Human.csv")
dataset = dataset["train"]

dataset = dataset.shuffle(seed=42).select(range(20000))

split_dataset = dataset.train_test_split(test_size=0.2)
test_valid = split_dataset["test"].train_test_split(test_size=0.5)

final_splits = {
    "train": split_dataset["train"],
    "validation": test_valid["train"],
    "test": test_valid["test"],
}

dataset_dict = final_splits

In [3]:
dataset_dict

{'train': Dataset({
     features: ['text', 'generated'],
     num_rows: 16000
 }),
 'validation': Dataset({
     features: ['text', 'generated'],
     num_rows: 2000
 }),
 'test': Dataset({
     features: ['text', 'generated'],
     num_rows: 2000
 })}

In [4]:
from datasets import DatasetDict
dataset_dict = DatasetDict(dataset_dict)  # Convert it to DatasetDict

In [5]:
dataset_dict = dataset_dict.rename_column("generated", "labels")


In [6]:
dataset_dict['train'].features

{'text': Value(dtype='string', id=None),
 'labels': Value(dtype='float64', id=None)}

In [7]:
dataset_dict

DatasetDict({
    train: Dataset({
        features: ['text', 'labels'],
        num_rows: 16000
    })
    validation: Dataset({
        features: ['text', 'labels'],
        num_rows: 2000
    })
    test: Dataset({
        features: ['text', 'labels'],
        num_rows: 2000
    })
})

In [8]:
def convert_labels(example):
    example["labels"] = int(example["labels"])
    return example

dataset_dict = dataset_dict.map(convert_labels)


Map: 100%|██████████| 2000/2000 [00:00<00:00, 21994.94 examples/s]


In [9]:
from collections import Counter
print(Counter(dataset_dict["train"]["labels"]))  # Count occurrences of each label

Counter({0.0: 10004, 1.0: 5996})


In [10]:
np.shape(dataset_dict["train"]["labels"])

(16000,)

In [11]:
np.shape(dataset_dict["train"]["text"])

(16000,)

In [12]:
dataset_dict["train"]["text"][0]

"Is it possible for younger and older students to work together? Older students are smarter than younger students, because they have more experience than the younger students. Also, the older students could help the younger students with their school work. Schools should have a program that Paris older students with younger students because they could help each other with school works, help them learn more about team work, and they could learn some viable things and give each other great advice when they need it.\n\nHow could older students help younger students with school work. Older students are smarter than younger students, whereas younger students could learn faster than older students. My older sister is really smart every time I ask for help she always helps me, and she shows me how to solve the problems that I don't understand. After she shows me how to solve the problem I would understand the question and solve the other problems. That is how older students could help younger

In [13]:
model_path = "google-bert/bert-base-uncased"

tokenizer = AutoTokenizer.from_pretrained(model_path)

id2label = {0: "Human", 1: "AI"}
label2id = {"Human": 0, "AI": 1}
model = AutoModelForSequenceClassification.from_pretrained(model_path, num_labels=2, id2label=id2label, label2id=label2id)
model.config.problem_type = "single_label_classification"

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at google-bert/bert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [14]:
#layers
for name, param in model.named_parameters():
   print(name, param.requires_grad)

bert.embeddings.word_embeddings.weight True
bert.embeddings.position_embeddings.weight True
bert.embeddings.token_type_embeddings.weight True
bert.embeddings.LayerNorm.weight True
bert.embeddings.LayerNorm.bias True
bert.encoder.layer.0.attention.self.query.weight True
bert.encoder.layer.0.attention.self.query.bias True
bert.encoder.layer.0.attention.self.key.weight True
bert.encoder.layer.0.attention.self.key.bias True
bert.encoder.layer.0.attention.self.value.weight True
bert.encoder.layer.0.attention.self.value.bias True
bert.encoder.layer.0.attention.output.dense.weight True
bert.encoder.layer.0.attention.output.dense.bias True
bert.encoder.layer.0.attention.output.LayerNorm.weight True
bert.encoder.layer.0.attention.output.LayerNorm.bias True
bert.encoder.layer.0.intermediate.dense.weight True
bert.encoder.layer.0.intermediate.dense.bias True
bert.encoder.layer.0.output.dense.weight True
bert.encoder.layer.0.output.dense.bias True
bert.encoder.layer.0.output.LayerNorm.weight True


In [15]:
# freeze base model params
for name, param in model.base_model.named_parameters():
    param.requires_grad = False

# unfreeze base model pooling layers
for name, param in model.base_model.named_parameters():
    if "pooler" in name:
        param.requires_grad = True

In [16]:
for name, param in model.named_parameters():
   print(name, param.requires_grad)

bert.embeddings.word_embeddings.weight False
bert.embeddings.position_embeddings.weight False
bert.embeddings.token_type_embeddings.weight False
bert.embeddings.LayerNorm.weight False
bert.embeddings.LayerNorm.bias False
bert.encoder.layer.0.attention.self.query.weight False
bert.encoder.layer.0.attention.self.query.bias False
bert.encoder.layer.0.attention.self.key.weight False
bert.encoder.layer.0.attention.self.key.bias False
bert.encoder.layer.0.attention.self.value.weight False
bert.encoder.layer.0.attention.self.value.bias False
bert.encoder.layer.0.attention.output.dense.weight False
bert.encoder.layer.0.attention.output.dense.bias False
bert.encoder.layer.0.attention.output.LayerNorm.weight False
bert.encoder.layer.0.attention.output.LayerNorm.bias False
bert.encoder.layer.0.intermediate.dense.weight False
bert.encoder.layer.0.intermediate.dense.bias False
bert.encoder.layer.0.output.dense.weight False
bert.encoder.layer.0.output.dense.bias False
bert.encoder.layer.0.output.Lay

In [17]:
import re
import string

def preprocess_and_tokenize(examples):
    preprocessed_texts = []
    for text in examples["text"]:
        text = text.lower()
        text = re.sub(r"\d+", "", text)
        text = text.translate(str.maketrans("", "", string.punctuation))
        text = text.strip()
        preprocessed_texts.append(text)
        
    encoding = tokenizer(preprocessed_texts, padding="max_length", truncation=True)
    encoding["labels"] = [int(label) for label in examples["labels"]]
    return encoding

tokenized_data = dataset_dict.map(preprocess_and_tokenize, batched=True)

Map: 100%|██████████| 2000/2000 [00:00<00:00, 2237.31 examples/s]


In [18]:

accuracy = evaluate.load("accuracy")
auc_score = evaluate.load("roc_auc")

def compute_metrics(eval_pred):
    
    predictions, labels = eval_pred
    
    
    probabilities = np.exp(predictions) / np.exp(predictions).sum(-1, keepdims=True)
    
    positive_class_probs = probabilities[:, 1]
    
    auc = np.round(auc_score.compute(prediction_scores=positive_class_probs, references=labels)['roc_auc'],3)
    
    predicted_classes = np.argmax(predictions, axis=1)
    acc = np.round(accuracy.compute(predictions=predicted_classes, references=labels)['accuracy'],3)
    
    return {"Accuracy": acc, "AUC": auc}

Training Using CUDA

In [19]:
def collate_fn(batch):
        batch = {k: torch.tensor([d[k] for d in batch]) for k in batch[0]}
        batch["labels"] = batch["labels"].long()
        # print(batch)
        return batch
data_collator = DefaultDataCollator()

In [20]:
import torch
print(torch.cuda.is_available())
print(torch.cuda.get_device_name(0))

True
NVIDIA GeForce RTX 3050 Ti Laptop GPU


In [21]:
lr = 2e-4
batch_size = 50
num_epochs = 5

training_args = TrainingArguments(
    output_dir="bert-ai-classifier_teacher",
    learning_rate=lr,
    per_device_train_batch_size=batch_size,
    per_device_eval_batch_size=batch_size,
    num_train_epochs=num_epochs,
    logging_strategy="epoch",
    eval_strategy="epoch",
    save_strategy="epoch",
    load_best_model_at_end=True,
)
model.to("cuda")

BertForSequenceClassification(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(30522, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0-11): 12 x BertLayer(
          (attention): BertAttention(
            (self): BertSdpaSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm((768,), eps=1e

In [22]:
trainer = Trainer(
    model=model.to("cuda"),
    args=training_args,
    train_dataset=tokenized_data["train"],
    eval_dataset=tokenized_data["validation"],
    tokenizer=tokenizer,
    data_collator=collate_fn,
    compute_metrics=compute_metrics,
)


trainer.train()

C:\Users\jai54\AppData\Local\Temp\ipykernel_15872\1966093435.py:1: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(


Epoch,Training Loss,Validation Loss,Accuracy,Auc
1,0.223700,0.170985,0.932000,0.990000
2,0.141500,0.103810,0.966000,0.993000
3,0.128900,0.180976,0.922000,0.994000
4,0.123600,0.133107,0.948000,0.994000
5,0.119000,0.120313,0.956000,0.995000


TrainOutput(global_step=1600, training_loss=0.14733776092529297, metrics={'train_runtime': 3828.9021, 'train_samples_per_second': 20.894, 'train_steps_per_second': 0.418, 'total_flos': 2.10488844288e+16, 'train_loss': 0.14733776092529297, 'epoch': 5.0})

In [23]:
# apply model to validation dataset
predictions = trainer.predict(tokenized_data["train"])

logits = predictions.predictions
labels = predictions.label_ids

metrics = compute_metrics((logits, labels))
print(metrics)

{'Accuracy': np.float64(0.959), 'AUC': np.float64(0.992)}


In [24]:
import torch
from transformers import AutoTokenizer

model.eval()
def aipred(sentence):
    tokenizer = AutoTokenizer.from_pretrained("bert-base-uncased")
    
    inputs = tokenizer(sentence, padding=True, truncation=True, return_tensors="pt")
    inputs = {key: val.to("cuda") for key, val in inputs.items()}  # Move tensors to CUDA
    
    with torch.no_grad():
        outputs = model(**inputs)
    
    logits = outputs.logits
    probs = torch.nn.functional.softmax(logits, dim=-1)
    predicted_label = torch.argmax(probs, dim=-1).item()
    
    print(f"{((probs.data[0][1])*100):.2f}% AI Generated")

In [25]:
aipred("")

66.42% AI Generated
